
[上一篇文章](https://zhuanlan.zhihu.com/p/44584551)里安利了这个非常惊艳的关于红楼梦的可视化作品：[InteractiveGraph/example1](https://bluejoe2008.github.io/InteractiveGraph/dist/examples/example1.html)  
![](https://upload-images.jianshu.io/upload_images/7145358-d44cc2bdd8193db9.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

但关系图谱本身还是很复杂的，因而本文将进行简单的分析挖掘，来增进对数据集了解；另外，最近接触了些依存句法分析、信息抽取、事件图谱等知识（后续会写写这方面内容），对实际项目中如何从非结构化的文本内容中抽取出结构化的数据感兴趣，看看能否在逆向过程中得到构建数据的启示。

## 数据说明
在[README_CN.md](https://github.com/bluejoe2008/InteractiveGraph/blob/master/README_CN.md)里关于数据集(Datasets)的说明：
> [honglou.json](https://github.com/bluejoe2008/InteractiveGraph/blob/master/dist/examples/honglou.json ) 数据集来自于中国古典名著《红楼梦》（又名《石头记》，https://en.wikipedia.org/wiki/Dream_of_the_Red_Chamber ）。 在这部小说中贾宝玉、林黛玉、薛宝钗是主要人物。这个数据集中定义了超过300个实体，其中包括书中的人物，地点和时间，以及超过500个这些实体之间的连接。
> nickel2008@github 提供了数据集。此数据集中或有纰漏，但是对于一个图数据项目的示例来说已经足够好了。

疑似相关的两个项目，后续进一步验证：
- https://github.com/lzell/nickel  
- https://github.com/iainbeeston/nickel

## 数据准备
虽然该数据集不是从头开始构建的，但不妨碍继续进行探索分析。复制粘贴原数据集`honglou.json`到本地，并删除掉下面无用的代码，这样后续读取json数据时才不会出错。最后记得保存成`utf-8`编码格式。
```
"translator": {
    "nodes": function (node) {
      //set description
      if (node.description === undefined) {
        var description = "<p align=center>";
        if (node.image !== undefined) {
          description += "<img src='" + node.image + "' width=150/><br>";
        }
        description += "<b>" + node.label + "</b>" + "[" + node.id + "]";
        description += "</p>";
        if (node.info !== undefined) {
          description += "<p align=left>" + node.info + "</p>";
        } else {
          if (node.title !== undefined)
            description += "<p align=left>" + node.title + "</p>";
        }
        node.description = description;
      }
    },
  },
```

## 读取数据

In [1]:
import json
import codecs

with codecs.open('InteractiveGraph_HongLouMeng.json', 'r',encoding='utf-8') as json_str:
    json_dict = json.load(json_str)
    print(json_dict.keys())
    print(json_dict["categories"].keys())
    print(json_dict["data"].keys())
    print(json_dict["categories"])
    nodes = json_dict['data']['nodes']
    edges = json_dict['data']['edges']

dict_keys(['categories', 'data'])
dict_keys(['person', 'event', 'location'])
dict_keys(['nodes', 'edges'])
{'person': '人物', 'event': '事件', 'location': '地点'}


层级关系大致如此，`categories`和`data`同一级，节点`nodes`和边`edges`同一级，并且归属于`data`，也是本次要统计分析的所有数据，`categories`指明三种节点数据类型，即：`'person': '人物', 'event': '事件', 'location': '地点`。 

## 红楼多少事
首先来看看数据中都包含了哪些红楼梦中的事件，直接筛选出类型为`event`的节点，共拿到59条数据。
```
{
        "label": "共读西厢",
        "value": 2,
        "id": 3779,
        "categories": [
          "event"
        ],
        "info": "宝玉到沁芳桥边桃花底下看《西厢记》，正准备将落花送进池中，黛玉说她早已准备了一个花冢，正来葬花。黛玉发现《西厢记》，宝玉借书中词句，向黛玉表白。黛玉觉得冒犯了自己尊严，引起口角，宝玉赔礼讨饶，黛玉也借《西厢记》词句，嘲笑了宝玉。于是两人收拾落花，葬到花冢里去。"
      },
```

In [2]:
event_nodes = []
for num, node in enumerate(nodes):
    if node['categories'][0] == 'event':
        event_nodes.append(node)
print(len(event_nodes))

59


字典元素组成的列表直接用`pandas`转成表格格式。 

其中`label`就是事件名称，`info`是内容简介，`value`貌似是觉得节点大小的，未做细究，本次均不做探索。

In [3]:
import pandas as pd
df = pd.DataFrame(event_nodes)
df.head()

,categories,id,info,label,value
0,[event],3779,宝玉到沁芳桥边桃花底下看《西厢记》，正准备将落花送进池中，黛玉说她早已准备了一个花冢，正来葬...,共读西厢,2
1,[event],3780,林如海考中探花后，迁为兰台寺大夫，钦点为扬州巡盐御史。后身染重病于九月初三日巳时而亡。,林如海捐馆扬州城,4
2,[event],3781,初秋季节，贾探春提议邀集大观园中有文采的人所组的诗社。诗社成立目的 旨在“宴集诗人於风庭月榭...,海棠诗社,8
3,[event],3782,紫鹃想试试宝玉对黛玉的情意，就单独骗他说自己要和黛玉回苏州，宝玉本来不信，但在紫鹃一番骗小孩...,紫鹃试玉,2
4,[event],3783,贾环用油灯烫了宝玉后，王夫人骂了赵姨娘没好好管教。赵姨娘受气后，正巧马道婆来和她闲聊。赵姨娘...,魇魔姊弟,6


将事件全部提取出来：

In [4]:
events = df['label'].values.tolist()
events

['共读西厢',
 '林如海捐馆扬州城',
 '海棠诗社',
 '紫鹃试玉',
 '魇魔姊弟',
 '羞笼红麝串',
 '麒麟伏双星',
 '纳鸳鸯',
 '撵晴雯',
 '偷娶尤二姐',
 '软语救贾琏',
 '大闹学堂',
 '拐卖巧姐',
 '乱判葫芦案',
 '毒设相思局',
 '情赠茜香罗',
 '勇救薛蟠',
 '倪二轻财尚义',
 '神游太虚幻境',
 '借剑杀人',
 '平儿失镯',
 '平儿行权',
 '司棋被捉',
 '巧结梅花络',
 '亲尝莲叶羹',
 '宝玉挨打',
 '大闹厨房',
 '香菱学诗',
 '凤姐托孤',
 '旺儿妇霸成亲',
 '弄权铁槛寺',
 '智能偷情',
 '勾引薛蝌',
 '贾政借钱',
 '探春远嫁',
 '刘姥姥一进荣国府',
 '黛玉葬花',
 '宝钗扑蝶',
 '金钏投井',
 '大观园试才',
 '秦可卿淫丧天香楼',
 '迎春误嫁中山狼',
 '金玉良缘',
 '王熙凤协理宁国府',
 '元妃省亲',
 '甄士隐梦幻识通灵',
 '晴雯撕扇',
 '凤姐泼醋',
 '探春理家',
 '湘云醉眠芍药裀',
 '尤三姐殉情',
 '抄检大观园',
 '黛玉焚稿',
 '黛玉之死',
 '晴雯补裘',
 '元宵丢英莲',
 '冷子兴演说荣国府',
 '木石前盟',
 '贤袭人娇嗔箴宝玉']

拿到这些事件后下一步该怎么办？让我们再明确下本文的目的之一，即看看能否逆向找出数据构造的规则与逻辑。那么自然而然的就有一个问题：这些事件都是如何从原著中抽取出来或者总结出来的呢？ 

![](https://upload-images.jianshu.io/upload_images/7145358-9edd18a0ffd8e76a.jpg?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

作为中国古典四大名著之首的《红楼梦》，有1600余页、73万余字（人民文学出版社版本），涉及的人物和事件繁多，若是单纯靠人工去总结，显然并不可取，而且也无法迁移到其他文本上去。当然，《红楼梦》本身广受读者喜爱，历来研究的人也多，且妇孺皆知、耳熟能详，网上现成的人物名单、事件罗列，想来或多或少都是有的，此处暂且不表。

考虑到《红楼梦》本身是章回体小说，各章回的名字高度总结概括了本章的内容，一个合理的猜想就是从章回中直接抽取出事件内容。那么就来看看这59条数据里有多少是完全和章回名重合的呢？
![](https://upload-images.jianshu.io/upload_images/7145358-3cacd88a99a26d15.jpg?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

## 获取章节名
首先从[《红楼梦》小说章节目录](https://www.555zw.com/book/39/39480/)网站获取各章回名称，简单写个爬虫就行。

In [5]:
import requests
from lxml import etree

url = 'https://www.555zw.com/book/39/39480/'
r = requests.get(url)
print(r.content[:600])

b'<!DOCTYPE html><head>\n<title>\xba\xec\xc2\xa5\xc3\xce\xd7\xee\xd0\xc2\xd5\xc2\xbd\xda(\xc8\xab\xb1\xbe\xb4\xf3\xbd\xe1\xbe\xd6)\xa3\xac\xba\xec\xc2\xa5\xc3\xce\xc8\xab\xce\xc4\xd4\xc4\xb6\xc1\xa3\xac\xba\xec\xc2\xa5\xc3\xce\xce\xde\xb5\xaf\xb4\xb0\xa3\xac\xd7\xf7\xd5\xdf\xa3\xba\xb2\xdc\xd1\xa9\xc7\xdb - \xc8\xfd\xce\xe5\xd6\xd0\xce\xc4\xcd\xf8</title>\n<meta http-equiv="Content-Type" content="text/html; charset=gbk" />\n<META HTTP-EQUIV="pragma" CONTENT="no-cache">\n<META HTTP-EQUIV="Cache-Control" CONTENT="no-store, must-revalidate">\n<META HTTP-EQUIV="expires" CONTENT="Wed, 26 Feb 1997 08:21:57 GMT">\n<META HTTP-EQUIV="expires" CONTENT="0">\n<meta name="description" content="\xc3\xe2\xb7\xd1\xd4\xc4\xb6\xc1\xd0\xa1\xcb\xb5\xba\xec\xc2\xa5\xc3\xce\xb5\xc4\xd7\xee\xd0\xc2\xd5\xc2\xbd\xda,\xb2\xdc\xd1\xa9\xc7\xdb\xb5\xc4\xba\xec\xc2\xa5\xc3\xce\xc4\xbf\xc2\xbc\xba\xcf\xbc\xaf\xc1\xd0\xb1\xed,\xc8\xfd\xce\xe5\xd6\xd0\xce\xc4\xcd\xf8\xa1\xa3" />\n<meta name="keywords" content="\xba\xe

返回的数据乱码，需要设置编码格式。 

[代码分析Python requests库中文编码问题](http://xiaorui.cc/2016/02/19/%E4%BB%A3%E7%A0%81%E5%88%86%E6%9E%90python-requests%E5%BA%93%E4%B8%AD%E6%96%87%E7%BC%96%E7%A0%81%E9%97%AE%E9%A2%98/)

In [6]:
print(r.apparent_encoding)

GB2312


In [7]:
import requests
from lxml import etree

url = 'https://www.555zw.com/book/39/39480/'
r = requests.get(url)
r.encoding = r.apparent_encoding # GB2312

selector = etree.HTML(r.content)
contents = selector.xpath('//tr//a/@title')
print(len(contents))
contents

120


['第一回 甄士隐梦幻识通灵 贾雨村风尘怀闺秀',
 '第二回 贾夫人仙逝扬州城 冷子兴演说荣国府',
 '第三回 贾雨村夤缘复旧职 林黛玉抛父进京都',
 '第四回 薄命女偏逢薄命郎 葫芦僧乱判葫芦案',
 '第五回 游幻境指迷十二钗 饮仙醪曲演红楼梦',
 '第六回 贾宝玉初试云雨情 刘姥姥一进荣国府',
 '第七回 送宫花贾琏戏熙凤 宴宁府宝玉会秦钟',
 '第八回 比通灵金莺微露意 探宝钗黛玉半含酸',
 '第九回 恋风流情友入家塾 起嫌疑顽童闹学堂',
 '第十回 金寡妇贪利权受辱 张太医论病细穷源',
 '第十一回 庆寿辰宁府排家宴 见熙凤贾瑞起淫心',
 '第十二回 王熙凤毒设相思局 贾天祥正照风月鉴',
 '第十三回 秦可卿死封龙禁尉 王熙凤协理宁国府',
 '第十四回 林如海捐馆扬州城 贾宝玉路谒北静王',
 '第十五回 王凤姐弄权铁槛寺 秦鲸卿得趣馒头庵',
 '第十六回 贾元春才选凤藻宫 秦鲸卿夭逝黄泉路',
 '第十七回 大观园试才题对额 荣国府归省庆元宵',
 '第十八回 隔珠帘父女勉忠勤 搦湘管姊弟裁题咏',
 '第十九回 情切切良宵花解语 意绵绵静日玉生香',
 '第二十回 王熙凤正言弹妒意 林黛玉俏语谑娇音',
 '第二十一回 贤袭人娇嗔箴宝玉 俏平儿软语救贾琏',
 '第二十二回 听曲文宝玉悟禅机 制灯迷贾政悲谶语',
 '第二十三回 西厢记妙词通戏语 牡丹亭艳曲警芳心',
 '第二十四回 醉金刚轻财尚义侠 痴女儿遗帕惹相思',
 '第二十五回 魇魔法姊弟逢五鬼 红楼梦通灵遇双真',
 '第二十六回 蜂腰桥设言传心事 潇湘馆春困发幽情',
 '第二十七回 滴翠亭杨妃戏彩蝶 埋香冢飞燕泣残红',
 '第二十八回 蒋玉菡情赠茜香罗 薛宝钗羞笼红麝串',
 '第二十九回 享福人福深还祷福 痴情女情重愈斟情',
 '第三十回 宝钗借扇机带双敲 龄官划蔷痴及局外',
 '第三十一回 撕扇子作千金一笑 因麒麟伏白首双星',
 '第三十二回 诉肺腑心迷活宝玉 含耻辱情烈死金钏',
 '第三十三回 手足耽耽小动唇舌 不肖种种大承笞挞',
 '第三十四回 情中情因情感妹妹 错里错以错劝哥哥',
 '第三十五回 白玉钏亲尝莲叶羹 黄金莺巧结梅花络',
 '第三十六回 绣鸳鸯梦兆绛芸轩 识分定情悟梨香院',
 '第三十七回 秋爽斋偶结海棠社 蘅芜苑夜拟菊

小bug：`'第七十二回\u3000王熙凤恃强羞说病\u3000来旺妇倚势霸成亲'`

In [8]:
contents[71] = contents[71].replace('\u3000', ' ')
contents

['第一回 甄士隐梦幻识通灵 贾雨村风尘怀闺秀',
 '第二回 贾夫人仙逝扬州城 冷子兴演说荣国府',
 '第三回 贾雨村夤缘复旧职 林黛玉抛父进京都',
 '第四回 薄命女偏逢薄命郎 葫芦僧乱判葫芦案',
 '第五回 游幻境指迷十二钗 饮仙醪曲演红楼梦',
 '第六回 贾宝玉初试云雨情 刘姥姥一进荣国府',
 '第七回 送宫花贾琏戏熙凤 宴宁府宝玉会秦钟',
 '第八回 比通灵金莺微露意 探宝钗黛玉半含酸',
 '第九回 恋风流情友入家塾 起嫌疑顽童闹学堂',
 '第十回 金寡妇贪利权受辱 张太医论病细穷源',
 '第十一回 庆寿辰宁府排家宴 见熙凤贾瑞起淫心',
 '第十二回 王熙凤毒设相思局 贾天祥正照风月鉴',
 '第十三回 秦可卿死封龙禁尉 王熙凤协理宁国府',
 '第十四回 林如海捐馆扬州城 贾宝玉路谒北静王',
 '第十五回 王凤姐弄权铁槛寺 秦鲸卿得趣馒头庵',
 '第十六回 贾元春才选凤藻宫 秦鲸卿夭逝黄泉路',
 '第十七回 大观园试才题对额 荣国府归省庆元宵',
 '第十八回 隔珠帘父女勉忠勤 搦湘管姊弟裁题咏',
 '第十九回 情切切良宵花解语 意绵绵静日玉生香',
 '第二十回 王熙凤正言弹妒意 林黛玉俏语谑娇音',
 '第二十一回 贤袭人娇嗔箴宝玉 俏平儿软语救贾琏',
 '第二十二回 听曲文宝玉悟禅机 制灯迷贾政悲谶语',
 '第二十三回 西厢记妙词通戏语 牡丹亭艳曲警芳心',
 '第二十四回 醉金刚轻财尚义侠 痴女儿遗帕惹相思',
 '第二十五回 魇魔法姊弟逢五鬼 红楼梦通灵遇双真',
 '第二十六回 蜂腰桥设言传心事 潇湘馆春困发幽情',
 '第二十七回 滴翠亭杨妃戏彩蝶 埋香冢飞燕泣残红',
 '第二十八回 蒋玉菡情赠茜香罗 薛宝钗羞笼红麝串',
 '第二十九回 享福人福深还祷福 痴情女情重愈斟情',
 '第三十回 宝钗借扇机带双敲 龄官划蔷痴及局外',
 '第三十一回 撕扇子作千金一笑 因麒麟伏白首双星',
 '第三十二回 诉肺腑心迷活宝玉 含耻辱情烈死金钏',
 '第三十三回 手足耽耽小动唇舌 不肖种种大承笞挞',
 '第三十四回 情中情因情感妹妹 错里错以错劝哥哥',
 '第三十五回 白玉钏亲尝莲叶羹 黄金莺巧结梅花络',
 '第三十六回 绣鸳鸯梦兆绛芸轩 识分定情悟梨香院',
 '第三十七回 秋爽斋偶结海棠社 蘅芜苑夜拟菊

In [9]:
chapters = [cont.split(' ')[0] for cont in contents]
contents = [cont.split('回 ')[-1] for cont in contents]
print(len(contents))
# print(chapters)
# print(contents)

120


将事件文本包含在章回名的事件与章回分别提取出来

In [10]:
event_chaps = []
noval_chaps = []
for event in events:
    for chap in contents:
        if event in chap:
            event_chaps.append(event)
            noval_chaps.append(chap)
            
print(len(noval_chaps))           
print(len(set(noval_chaps)))
noval_chaps

21
18


['林如海捐馆扬州城 贾宝玉路谒北静王',
 '蒋玉菡情赠茜香罗 薛宝钗羞笼红麝串',
 '贤袭人娇嗔箴宝玉 俏平儿软语救贾琏',
 '薄命女偏逢薄命郎 葫芦僧乱判葫芦案',
 '王熙凤毒设相思局 贾天祥正照风月鉴',
 '蒋玉菡情赠茜香罗 薛宝钗羞笼红麝串',
 '弄小巧用借剑杀人 觉大限吞生金自逝',
 '投鼠忌器宝玉瞒赃 判冤决狱平儿行权',
 '白玉钏亲尝莲叶羹 黄金莺巧结梅花络',
 '白玉钏亲尝莲叶羹 黄金莺巧结梅花络',
 '王凤姐弄权铁槛寺 秦鲸卿得趣馒头庵',
 '贾宝玉初试云雨情 刘姥姥一进荣国府',
 '大观园试才题对额 荣国府归省庆元宵',
 '薛文龙悔娶河东狮 贾迎春误嫁中山狼',
 '秦可卿死封龙禁尉 王熙凤协理宁国府',
 '甄士隐梦幻识通灵 贾雨村风尘怀闺秀',
 '变生不测凤姐泼醋 喜出望外平儿理妆',
 '惑奸谗抄检大观园 矢孤介杜绝宁国府',
 '林黛玉焚稿断痴情 薛宝钗出闺成大礼',
 '贾夫人仙逝扬州城 冷子兴演说荣国府',
 '贤袭人娇嗔箴宝玉 俏平儿软语救贾琏']

重复项：  
```'蒋玉菡情赠茜香罗 薛宝钗羞笼红麝串',
'贤袭人娇嗔箴宝玉 俏平儿软语救贾琏,
'白玉钏亲尝莲叶羹 黄金莺巧结梅花络',```

In [11]:
print(len(event_chaps))
event_chaps

21


['林如海捐馆扬州城',
 '羞笼红麝串',
 '软语救贾琏',
 '乱判葫芦案',
 '毒设相思局',
 '情赠茜香罗',
 '借剑杀人',
 '平儿行权',
 '巧结梅花络',
 '亲尝莲叶羹',
 '弄权铁槛寺',
 '刘姥姥一进荣国府',
 '大观园试才',
 '迎春误嫁中山狼',
 '王熙凤协理宁国府',
 '甄士隐梦幻识通灵',
 '凤姐泼醋',
 '抄检大观园',
 '黛玉焚稿',
 '冷子兴演说荣国府',
 '贤袭人娇嗔箴宝玉']

In [12]:
chapter_df = pd.DataFrame({"chapter":chapters, "title":contents})
chapter_df.head()

,chapter,title
0,第一回,甄士隐梦幻识通灵 贾雨村风尘怀闺秀
1,第二回,贾夫人仙逝扬州城 冷子兴演说荣国府
2,第三回,贾雨村夤缘复旧职 林黛玉抛父进京都
3,第四回,薄命女偏逢薄命郎 葫芦僧乱判葫芦案
4,第五回,游幻境指迷十二钗 饮仙醪曲演红楼梦


In [13]:
def is_event(title):
    for event in event_chaps:
        if event in title:
            return event
    return ''
chapter_df['title2event'] = chapter_df['title'].apply(is_event)
chapter_df.head()

,chapter,title,title2event
0,第一回,甄士隐梦幻识通灵 贾雨村风尘怀闺秀,甄士隐梦幻识通灵
1,第二回,贾夫人仙逝扬州城 冷子兴演说荣国府,冷子兴演说荣国府
2,第三回,贾雨村夤缘复旧职 林黛玉抛父进京都,
3,第四回,薄命女偏逢薄命郎 葫芦僧乱判葫芦案,乱判葫芦案
4,第五回,游幻境指迷十二钗 饮仙醪曲演红楼梦,


In [15]:
# chapter_df

## 表格上色
美化pandas中DataFrame的输出  
[pandas-docs/style](http://pandas.pydata.org/pandas-docs/stable/style.html)

[在线取色器](http://link.fobshanghai.com/rgbcolor.htm)   
[马卡龙色系主题网页DEMO](https://www.zcool.com.cn/work/ZMTQ2NjgxMzI=.html)   
<img src='马卡龙色系主题网页DEMO.jpg'>

In [16]:
chapter_df[chapter_df.title2event != ''].style.set_properties(**{'background-color': '#ccff99',
                           'color': '#B452CD',
                           #'border-color': 'white'
                            })

,chapter,title,title2event
0,第一回,甄士隐梦幻识通灵 贾雨村风尘怀闺秀,甄士隐梦幻识通灵
1,第二回,贾夫人仙逝扬州城 冷子兴演说荣国府,冷子兴演说荣国府
3,第四回,薄命女偏逢薄命郎 葫芦僧乱判葫芦案,乱判葫芦案
5,第六回,贾宝玉初试云雨情 刘姥姥一进荣国府,刘姥姥一进荣国府
11,第十二回,王熙凤毒设相思局 贾天祥正照风月鉴,毒设相思局
12,第十三回,秦可卿死封龙禁尉 王熙凤协理宁国府,王熙凤协理宁国府
13,第十四回,林如海捐馆扬州城 贾宝玉路谒北静王,林如海捐馆扬州城
14,第十五回,王凤姐弄权铁槛寺 秦鲸卿得趣馒头庵,弄权铁槛寺
16,第十七回,大观园试才题对额 荣国府归省庆元宵,大观园试才
20,第二十一回,贤袭人娇嗔箴宝玉 俏平儿软语救贾琏,软语救贾琏


由上图可知，共有18条（18/59=30%）事件是一字不差包含在章回名里的。不过感觉非红迷的朋友，可能不熟悉这些事件到底是什么情节（是这样吗？）

## 非章节名的事件
将list列表转换成set集合，再求差集。 

In [17]:
s = set(conts) - set(event_chaps)
print(len(s))
s

NameError: name 'conts' is not defined

其他41条事件按人物角色和小说情节出现的前后顺序进行简单整理，比较耳熟能详的有：'木石前盟', '金玉良缘', '共读西厢', '宝钗扑蝶','黛玉葬花','晴雯撕扇', '湘云醉眠芍药裀', '香菱学诗'等等。

```{
 '元宵丢英莲', '木石前盟', '金玉良缘', '麒麟伏双星', '神游太虚幻境',  '秦可卿淫丧天香楼',
 '倪二轻财尚义', '智能偷情', '旺儿妇霸成亲',
 '大闹学堂', '宝玉挨打', '元妃省亲', '共读西厢', '宝钗扑蝶', '海棠诗社', '湘云醉眠芍药裀', '香菱学诗',
 '魇魔姊弟', '金钏投井', '紫鹃试玉', '大闹厨房', '司棋被捉',
 '晴雯撕扇', '晴雯补裘', '撵晴雯',
 '平儿失镯', '凤姐托孤', '拐卖巧姐',
 '探春理家', '探春远嫁', '黛玉葬花', '黛玉之死',
 '纳鸳鸯', '偷娶尤二姐', '尤三姐殉情',
 '贾政借钱', '勇救薛蟠', '勾引薛蝌',
}``` 

其中，`'宝钗扑蝶'和'黛玉葬花'`均对应`第二十七回 滴翠亭杨妃戏彩蝶 埋香冢飞燕泣残红`。可见还是可以转换成从章节名里提取事件的。
![](https://upload-images.jianshu.io/upload_images/7145358-2a82acbb17399da1.jpg?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

未用来提取事件的章回

In [18]:
chapter_df[chapter_df.title2event == '']

,chapter,title,title2event
2,第三回,贾雨村夤缘复旧职 林黛玉抛父进京都,
4,第五回,游幻境指迷十二钗 饮仙醪曲演红楼梦,
6,第七回,送宫花贾琏戏熙凤 宴宁府宝玉会秦钟,
7,第八回,比通灵金莺微露意 探宝钗黛玉半含酸,
8,第九回,恋风流情友入家塾 起嫌疑顽童闹学堂,
9,第十回,金寡妇贪利权受辱 张太医论病细穷源,
10,第十一回,庆寿辰宁府排家宴 见熙凤贾瑞起淫心,
15,第十六回,贾元春才选凤藻宫 秦鲸卿夭逝黄泉路,
17,第十八回,隔珠帘父女勉忠勤 搦湘管姊弟裁题咏,
18,第十九回,情切切良宵花解语 意绵绵静日玉生香,


以上就是对数据集中事件这一维度的分析，借助章回名和耳熟能详的桥段，可以拿到大多数事件。而有了事件后，如何提取事件中涉及的主要人物，这又是需要解决的，并且如何对其他不含章回名的、不那么熟悉的文本进行实体关系抽取、事件图谱构建等等都是需要进一步研究的。

## location 地点
接下来，看看`location`地点数据。格式如下：
```
{
        "label": "太虚幻境",
        "value": 1,
        "id": 3860,
        "categories": [
          "location"
        ],
        "info": "太虚幻境，《红楼梦》中的女儿仙境，警幻仙子司主。它位于离恨天之上、灌愁海之中的放春山遣香洞，以梦境的形式向甄士隐、贾宝玉二位有缘人显现。"
      },
```

In [19]:
loc_nodes = []
for num, node in enumerate(nodes):
    if node['categories'][0] == 'location':
        loc_nodes.append(node)
print(len(loc_nodes))

26


In [20]:
loc_df = pd.DataFrame(loc_nodes)
loc_df.head()

,categories,id,info,label,value
0,[location],3838,黛玉的居所。黛玉作诗的笔名就潇湘妃子，这是曹雪芹对黛玉这个人物的赞美。,潇湘馆,3
1,[location],3839,薛宝钗的住所，匾额题为“蘅芷清芬”。宝钗亦因此得诗号“蘅芜君”。蘅芜苑离正殿不远，在沁芳池边...,蘅芜苑,3
2,[location],3840,惜春的住所。暖香坞位于大观园的西半区，北面是李纨住的稻香村，南边是薛宝钗的蘅芜苑。,暖香坞,2
3,[location],3841,贾迎春的住所。大观园东部有一个庭院，西侧临水，东部靠山，院内西部建筑是紫菱洲，北房正厅，即缀锦楼。,缀锦楼,3
4,[location],3842,李纨的住所。转过山怀中，隐隐露出一带黄泥墙，墙上皆用稻茎掩护。有几百枝杏花，如喷火蒸霞一般。...,稻香村,3


In [21]:
print(loc_df.label.values.tolist())

['潇湘馆', '蘅芜苑', '暖香坞', '缀锦楼', '稻香村', '凤藻宫', '秋爽斋', '怡红院', '栊翠庵', '梨香院', '荣国府', '宁国府', '苏州', '大观园', '玄真观', '京郊', '扬州', '金陵', '京城', '胡州', '葫芦庙', '南海', '太虚幻境', '大同府', '阊门', '应天府']


本数据集给出的地点不算多，仅26条，主要是城市、贾府、大观园、各主要人物的住处等等。这部分可以用命名实体识别、或手动创建地点词典、或网上找现成的汇总等，应该能比较方便的实现，所以不展开了。至于人物与地点关系的抽取，同样不清楚有什么自动化的方式可以实现嘛？  

看到这些熟悉地名，也是想起自己曾去过北京和上海青浦南北两处大观园，网上盗张图，怀念一下：
![](https://upload-images.jianshu.io/upload_images/7145358-873dd98ea6cf33c6.jpg?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

## person 人物
再来看看`person`人物数据详情。格式如下：
```
{
        "label": "林黛玉",
        "value": 21,
        "image": "./images/photo/林黛玉.jpg",
        "id": 4037,
        "categories": [
          "person"
        ],
        "info": "金陵十二钗之冠（与宝钗并列）。林如海与贾敏之女，宝玉的姑表妹，寄居荣国府 。她生性孤傲，多愁善感，才思敏捷。她与宝玉真心相爱，是宝玉反抗封建礼教的同盟，是自由恋爱的坚定追求者。"
      },
```

In [22]:
person_nodes = []
for num, node in enumerate(nodes):
    if node['categories'][0] == 'person':
        person = node['label']
        person_nodes.append(node)
print(len(person_nodes))

242


In [23]:
person_df = pd.DataFrame(person_nodes)
person_df.head()

,categories,id,image,info,label,value
0,[person],3863,./images/photo/贾宝玉.jpg,荣国府衔玉而诞的公子，贾政与王夫人之次子，阖府捧为掌上明珠，对他寄予厚望，他却走上了叛逆之路...,贾宝玉,52
1,[person],3864,NaN,荣国公贾源是宁国公贾演的弟弟，贾源死后他的儿子贾代善袭了他的爵位。贾代善的夫人就是贾母（又称...,贾代善,8
2,[person],3865,NaN,贾演长子，贾敷、贾敬之父，京营节度使世袭一等神威将军。,贾代化,5
3,[person],3866,NaN,贾代化长子，贾敬之兄。八九岁夭折。,贾敷,4
4,[person],3867,./images/photo/贾敬.jpg,宁国公贾演的孙子，京营节度使世袭一等神威将军贾代化的次子，贾珍之父。是乙卯科进士，却一味好道...,贾敬,6


In [24]:
person_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 6 columns):
categories    242 non-null object
id            242 non-null int64
image         112 non-null object
info          242 non-null object
label         242 non-null object
value         242 non-null int64
dtypes: int64(2), object(4)
memory usage: 11.4+ KB


共242条人物数据，其中有112人附带了1987版《红楼梦》电视剧的角色剧照，照片统一存放在：[`dist\examples\images\photo`](https://github.com/bluejoe2008/InteractiveGraph/tree/master/dist/examples/images/photo)。

In [25]:
import numpy as np
person_df[~person_df.image.isnull()].head()

,categories,id,image,info,label,value
0,[person],3863,./images/photo/贾宝玉.jpg,荣国府衔玉而诞的公子，贾政与王夫人之次子，阖府捧为掌上明珠，对他寄予厚望，他却走上了叛逆之路...,贾宝玉,52
4,[person],3867,./images/photo/贾敬.jpg,宁国公贾演的孙子，京营节度使世袭一等神威将军贾代化的次子，贾珍之父。是乙卯科进士，却一味好道...,贾敬,6
5,[person],3868,./images/photo/贾赦.jpg,字恩侯，荣国公之孙，贾代善、贾母之长子，邢夫人的丈夫，贾琏、迎春的父亲，是个无耻之徒，他承袭...,贾赦,15
6,[person],3869,./images/photo/贾政.jpg,字存周，贾母和贾代善所生的次子，贾宝玉的父亲，林黛玉的舅舅，薛宝钗的姨父。他是除贾母外荣国府...,贾政,24
7,[person],3870,./images/photo/贾蓉.jpg,贾珍之子，是一个十六七岁的少年，面目清秀，身材俊俏，轻裘宝带，美服华冠的男子。他原为监生，妻...,贾蓉,11


陈晓旭版的林黛玉了解一下：
![](https://upload-images.jianshu.io/upload_images/7145358-c53abe9fb6b76a01.jpg?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)
用[百年百图の中国（1900-1999）：另类python爬虫和PIL拼图](https://zhuanlan.zhihu.com/p/29864330) 一文里的代码将所有图片拼到一起看看。里面混入了一个奇怪的东西（黑白的那张）。

![](https://upload-images.jianshu.io/upload_images/7145358-d2fb3d7feaa52f51.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240) 

另外，尤三姐的照片搞错成了尤二姐，于是有两张尤二姐的，即第四行倒数第三四张（一位“红迷”的自我修养）。 

没有照片的角色

In [26]:
person_df[person_df.image.isnull()].head()

,categories,id,image,info,label,value
1,[person],3864,NaN,荣国公贾源是宁国公贾演的弟弟，贾源死后他的儿子贾代善袭了他的爵位。贾代善的夫人就是贾母（又称...,贾代善,8
2,[person],3865,NaN,贾演长子，贾敷、贾敬之父，京营节度使世袭一等神威将军。,贾代化,5
3,[person],3866,NaN,贾代化长子，贾敬之兄。八九岁夭折。,贾敷,4
12,[person],3875,NaN,贾赦之子。,贾琮,4
13,[person],3876,NaN,贾政嫡长子，与元春、宝玉均为王夫人所生，娶妻李纨，育子贾兰，在《红楼梦》中是作为已故人物。,贾珠,12


In [27]:
person_df[person_df['label'].isin(['尤二姐', '尤三姐'])]

,categories,id,image,info,label,value
150,[person],4080,./images/photo/尤二姐.jpg,贾琏的二房，是贾珍夫人尤氏的继母带来的女儿。尤二姐模样标致，温柔和顺。贾珍馋涎妻妹的美貌，对...,尤二姐,7
151,[person],4081,./images/photo/尤三姐.jpg,尤氏继母带来的女儿，尤二姐的妹妹，亦称作尤小妹。尤三姐模样儿风流标致，她又偏爱打扮得出色，自...,尤三姐,5


## edges 边
最后再来看看人物与人物、人物与地点、人物与事件的关系。数据格式：
```
"edges": [{
        "id": 3776,
        "label": "位于",
        "from": 3838,
        "to": 3851
      },
      {
        "id": 3777,
        "label": "位于",
        "from": 3839,
        "to": 3851
      },
```

In [30]:
len(edges)

694

In [31]:
edges_df = pd.DataFrame(edges)
edges_df.head()

,from,id,label,to
0,3838,3776,位于,3851
1,3839,3777,位于,3851
2,3840,3778,位于,3851
3,3841,3779,位于,3851
4,3842,3780,位于,3851


共25类694条数据。

In [37]:
edges_rela = edges_df.label.value_counts()
edges_rela

参与     187
仆人      80
居住地     61
父亲      54
原籍      42
母亲      39
丈夫      38
妻子      34
哥哥      26
交好      23
位于      15
同宗      13
姐姐      12
私通      12
老师      10
姬妾      10
喜欢       9
跟班       8
干娘       5
奶妈       5
知己       3
陪房       3
连宗       2
前世       2
有恩       1
Name: label, dtype: int64

用`pyecharts`绘制各类关系及其数量的柱形图。
![](https://upload-images.jianshu.io/upload_images/7145358-8dc0e11bf4d4205a.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)
最近python交友娱乐会所群（QQ：613176398）里看到很多人都也在用这个库，不过我又想重新用ECharts来“美颜”图表了，以往整理过的代码和示例可见：[图表太丑怎么破，ECharts神器带你飞！](https://zhuanlan.zhihu.com/p/29988515)。这里也用一下，颜值碾压。
![](https://upload-images.jianshu.io/upload_images/7145358-1be7ed5c8c4b6670.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)    

[彩色柱形图](http://gallery.echartsjs.com/editor.html?c=xBJSSvnu2b&v=2)  

In [38]:
edges_rela.index

Index(['参与', '仆人', '居住地', '父亲', '原籍', '母亲', '丈夫', '妻子', '哥哥', '交好', '位于', '同宗',
       '姐姐', '私通', '老师', '姬妾', '喜欢', '跟班', '干娘', '奶妈', '知己', '陪房', '连宗', '前世',
       '有恩'],
      dtype='object')

In [39]:
edges_rela.values

array([187,  80,  61,  54,  42,  39,  38,  34,  26,  23,  15,  13,  12,
        12,  10,  10,   9,   8,   5,   5,   3,   3,   2,   2,   1],
      dtype=int64)

In [40]:
from pyecharts import Bar
bar = Bar("edges 边关系")
bar.add("edges 边关系", edges_rela.index, edges_rela.values, is_stack=True,is_label_show=True,xaxis_interval=0,xaxis_rotate=-45,xaxis_margin=8)
bar

在这些关系中，首先看到了“私通”二字，那么就来看下都是谁和谁私通吧。写成函数方便复用。这里`edges`只包含相关节点的`id`，需要从`person`里拿到对应的人物名称。

In [42]:
st = edges_df[edges_df.label=='私通']
st

,from,id,label,to
105,3871,3274,私通,4140
119,3873,3275,私通,4053
133,3874,3276,私通,4122
187,3885,3277,私通,4068
404,4041,3579,私通,3870
516,4053,3580,私通,3871
555,4063,3581,私通,3927
569,4068,3582,私通,3885
607,4081,3583,私通,3873
621,4086,3584,私通,3874


In [43]:
st_from = st['from'].values.tolist()
st_to = st['to'].values.tolist()
print(len(st_from), len(st_to))
print(st_from)
print(st_to)

12 12
[3871, 3873, 3874, 3885, 4041, 4053, 4063, 4068, 4081, 4086, 4147, 4149]
[4140, 4053, 4122, 4068, 3870, 3871, 3927, 3885, 3873, 3874, 3881, 3926]


[Pandas速查手册中文版](https://zhuanlan.zhihu.com/p/25630700)   
 
[fastai/DL-lec4notes](http://forums.fast.ai/t/deeplearning-lec4notes/8146)   

[pandas-docs/merging](https://pandas.pydata.org/pandas-docs/stable/merging.html)  

In [44]:
def word2id(word):
    df = edges_df[edges_df.label== word]
    from_id = df['from'].values.tolist()
    to_id = df['to'].values.tolist()
    return from_id, to_id

def id2label(ids):
    tables = []
    for ID in ids:
        tables.append(person_df[person_df['id']==ID])
    labels = pd.concat(tables)['label'].values.tolist()
    return labels

def get_relation(from_id,to_id):
    for from_label, to_label in zip(id2label(from_id), id2label(to_id)):
        print(from_label, '--> {} -->'.format(word), to_label)

word = "私通"
from_id,to_id = word2id(word)
get_relation(from_id,to_id)

贾蔷 --> 私通 --> 龄官
贾珍 --> 私通 --> 秦可卿
贾琏 --> 私通 --> 多姑娘
薛蟠 --> 私通 --> 宝蟾
王熙凤 --> 私通 --> 贾蓉
秦可卿 --> 私通 --> 贾蔷
司棋 --> 私通 --> 潘又安
宝蟾 --> 私通 --> 薛蟠
尤三姐 --> 私通 --> 贾珍
鲍二家的 --> 私通 --> 贾琏
智能儿 --> 私通 --> 秦钟
万儿 --> 私通 --> 茗烟


以上就是私通名单！《红楼梦》里蛮出名的一句话是焦大说的：**“爬灰的爬灰，养小叔子的养小叔子”**，不明真相的吃瓜群众可以自行搜索。

## BUG
其中，贾琏也就是王熙凤凤姐的丈夫，分别和多姑娘、鲍二家的有私情。这里不得不开个车，其实《红楼梦》里也有几个黄段子的，下面两则均出自[第二十一回 《贤袭人娇嗔箴宝玉 俏平儿软语救贾琏》](https://www.555zw.com/book/39/39480/8950808.html)： 
> 贾琏见她娇俏动情，便搂着求欢，被平儿夺手跑了，急的贾琏弯着腰恨道：“死促狭小滢妇！一定浪上人的火来，他又跑了。”平儿在窗外笑道：“我浪我的，谁叫你动火了？难道图你受用一回，叫他知道了，又不待见我。” 

下面这个更好笑，因为新版红楼梦电视剧把这部分拍成了拔火罐，也是佩服导演的“神来之笔”，为18岁以下青少年的心理健康出了一份力。可见：[为什么网上对于旧版《红楼梦》的评价比新版《红楼梦》好那么多，旧版红楼是否被过度神话？](https://www.zhihu.com/question/39362218/answer/82120116)
> 那个贾琏，只离了凤姐便要寻事，独寝了两夜，便十分难熬，便暂将小厮们内有清俊的选来出火。

![](https://upload-images.jianshu.io/upload_images/7145358-e2d2c1b48d02d310.jpg?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

言归正传，本以为这里出现了个BUG：`秦可卿 --> 私通 --> 贾蔷` 应该是`秦可卿 --> 私通 --> 贾珍`，但一搜真有这些猜想，也就随它去吧。 

另外在原著里秦可卿，乳名兼美，暗含兼有钗黛之美的意思，在宝玉梦游太虚幻境时，写到“其鲜艳妩媚，有似乎宝钗，风流袅娜，则又如黛玉”。也是金陵十二钗中最先去世的女子。
![](https://upload-images.jianshu.io/upload_images/7145358-b99eb60ce05ca878.jpg?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

https://www.kaggle.com/gaborfodor/dog-breed-pretrained-keras-models-lb-0-3

In [45]:
person_df[person_df['label']=='贾蔷'].values

array([[list(['person']), 3871, './images/photo/贾蔷.jpg',
        '宁国府的正派玄孙。他父母早亡，从小跟贾珍过活，比贾蓉生得还风流俊俏。虽然每日应名去上学，亦只不过虚掩眼目而已，仍旧是斗鸡走狗，赏花阅柳。他上有贾珍溺爱，下有贾蓉匡助，越发自大起来。后成了贾府小戏班梨香院的总管，与小旦龄官相好。',
        '贾蔷', 7]], dtype=object)

[python3-cookbook](https://python3-cookbook.readthedocs.io/zh_CN/latest/preface.html)

## 其他关系
### 喜欢

In [47]:
edges_rela.index

Index(['参与', '仆人', '居住地', '父亲', '原籍', '母亲', '丈夫', '妻子', '哥哥', '交好', '位于', '同宗',
       '姐姐', '私通', '老师', '姬妾', '喜欢', '跟班', '干娘', '奶妈', '知己', '陪房', '连宗', '前世',
       '有恩'],
      dtype='object')

In [48]:
word = "喜欢"
from_id,to_id = word2id(word)
get_relation(from_id,to_id)

林黛玉 --> 喜欢 --> 贾宝玉
薛宝钗 --> 喜欢 --> 贾宝玉
妙玉 --> 喜欢 --> 贾宝玉
秦可卿 --> 喜欢 --> 贾宝玉
彩云 --> 喜欢 --> 贾环
尤三姐 --> 喜欢 --> 柳湘莲
藕官 --> 喜欢 --> 菂官
彩霞 --> 喜欢 --> 贾环
龄官 --> 喜欢 --> 贾蔷


### 知己

In [49]:
word = "知己"
from_id,to_id = word2id(word)
get_relation(from_id,to_id)

林黛玉 --> 知己 --> 紫鹃
妙玉 --> 知己 --> 邢岫烟
史湘云 --> 知己 --> 林黛玉


```
"label": "知己",
"from": 4037, "林黛玉"
"to": 4058, "紫鹃"
"from": 4050, "妙玉"
"to": 4078, "邢岫烟"
"from": 4052, "史湘云"
"to": 4037, "林黛玉"
```

### 交好

In [50]:
word = "交好"
from_id,to_id = word2id(word)
get_relation(from_id,to_id)

贾宝玉 --> 交好 --> 秦钟
贾宝玉 --> 交好 --> 柳湘莲
贾宝玉 --> 交好 --> 蒋玉菡
贾宝玉 --> 交好 --> 北静王
贾蓉 --> 交好 --> 贾琏
贾蔷 --> 交好 --> 秦钟
秦钟 --> 交好 --> 香怜
薛蟠 --> 交好 --> 柳湘莲
薛蟠 --> 交好 --> 冯紫英
薛蟠 --> 交好 --> 金荣
柳湘莲 --> 交好 --> 秦钟
贾雨村 --> 交好 --> 冷子兴
蒋玉菡 --> 交好 --> 北静王
贾芸 --> 交好 --> 贾蔷
贾菌 --> 交好 --> 贾蓝
赖尚荣 --> 交好 --> 柳湘莲
癞头和尚 --> 交好 --> 跛足道人
晴雯  --> 交好 --> 麝月
袭人 --> 交好 --> 平儿
小红 --> 交好 --> 坠儿
瑞珠 --> 交好 --> 宝珠
柳嫂子 --> 交好 --> 芳官
马道婆 --> 交好 --> 赵姨娘


感觉挺多和自己想的不一样的。但也懒得管了。逃......
![](https://upload-images.jianshu.io/upload_images/7145358-ff35bc6dff40c808.jpg?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

## 小结
以上算是“简单”完成了对该数据集的探索和分析，其实到底该如何在新的本文上构造好可用的、靠谱的数据集依旧不得而知，后续后写写句法依存分析、信息抽取、事件图谱等等的文章，敬请期待。（马卡龙伏笔）
![](https://upload-images.jianshu.io/upload_images/7145358-1a13081ae9072216.jpg?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)
